In [18]:
import pandas as pd
import os
import warnings; warnings.filterwarnings('ignore')

In [3]:
image_folder_path = "C:/Users/louis/github_vs/113_data/"

years = os.listdir(image_folder_path)
post_chat = years[5]
pre_chat = years[6]
years = years[0:5]
print(years)

['2017', '2018', '2019', '2020', '2021']


In [4]:
def get_msg_info_data():
    df_all = pd.DataFrame(columns =['event_id', 'message_id', 'user_handle', 'sec_since_start']);
    for y in years:
        
        events =  os.listdir(image_folder_path+y)

        for e in events:
            df = pd.read_csv(image_folder_path+y+'/'+e+'/'+"message_info.tsv",sep= '\t')
            df_all = df_all.merge(df, how = "outer")
            

    df_all.drop(df_all[df_all['user_handle'].str.contains('system')].index, inplace = True)

    df_all["id"] = df_all["event_id"].astype("string") + df_all["message_id"].astype("string")
    df_all.drop(['message_id'],axis=1,inplace=True)
    df_all=df_all.set_index("event_id",drop=False)
    df_all['user_handle'] = df_all['user_handle'].replace("employee.*","employee",regex = True)
    return df_all
df_all = get_msg_info_data()

In [5]:
# fusion with prechat dataset
def combine_with_chatquestions_data(df_all,df_quest):
    df_quest = df_quest.set_index("event_id")
    df_all = df_quest.join(df_all,how='inner')

    df_all.set_index('id',inplace=True)
    pos = df_all['Ik heb de wil om te leven']
    neg =  df_all.drop(['Ik heb de wil om te leven','user_handle','sec_since_start'],axis=1).mean(axis=1)
    df_all = pd.DataFrame({"postive":pos,"negative":neg,'user_handle':df_all['user_handle'],'sec_since_start':df_all['sec_since_start'],'event_id':df_all['event_id']})

    return df_all

df_pre = pd.read_csv(image_folder_path+"prechat_questions.tsv",sep= '\t').dropna()
df_all = combine_with_chatquestions_data(df_all,df_pre)

In [32]:
# include cds data in the Dataframe

def combine_with_cds_data(df_all):
    
    df = pd.read_csv(image_folder_path+years[4]+'/'+"202101"+'/'+"CDS_results.tsv",sep= '\t')
    cds_names =df.columns[2:].to_list()
    # cds that have less then 50 occurences in total over all the conversations, extracted from EDA phase3
    common_cds = ['hij zal( .+ | )niet denken', 'w(ij|e) zullen( .+ | )weten', 'een onbekwa(am|me)', 'w(ij|e) zullen( .+ | )niet geloven', 'een niet geliefd', 'zwart-/wit', 'w(ij|e) zullen( .+ | )niet weten', 'w(ij|e) zullen( .+ | )niet denken', 'zal( .+ | )verkeerd gaan', 'j(ij|e) (zal|zult)( .+ | )niet weten', 'j(ij|e) (zal|zult)( .+ | )niet denken', 'zal onmogelijk zijn', '(zal|gaat) niet plaatsvinden', 'wordt vreselijk', 'zal vreselijk zijn', 'wordt verschrikkelijk', 'zal verschrikkelijk zijn', 'wordt een ramp', 'zal( .+ | )een ramp zijn', '(zal|gaat) nooit eindigen', 'we zullen( .+ | ) niet', 'j(ij|e) (zal|zult)( .+ | ) niet', 'w(ij|e) zullen( .+ | )geloven', 'z(ij|e) zullen( .+ | )niet denken', 'z(ij|e) zullen( .+ | ) niet', 'afdoende,? maar', 'iedereen zal( .+ | )geloven', 'hij zal( .+ | )niet geloven', 'hij zal( .+ | )geloven', 'ik zie slechts', 'hij zal( .+ | )niet weten', 'zal( .+ | )mis gaan', 'eindigt niet', 'acceptabel,? maar', 'z(ij|e) zal( .+ | )niet denken', 'alleen het (aller)?ergste', 'alleen het slechtste', 'w(ij|e) zullen( .+ | )denken', 'j(ij|e) (zal|zult)( .+ | )niet geloven', 'ik veroorzaakte', 'z(ij|e) zal( .+ | )niet weten', 'wordt onmogelijk', '(zal|gaat) niet ophouden', 'j(ij|e) (zal|zult)( .+ | )geloven', 'ik zal( .+ | ) niet', 'een incompetente?', 'z(ij|e) zullen( .+ | )niet weten', 'z(ij|e) zal( .+ | )niet geloven', 'iedereen zal( .+ | )weten', 'een beschadigde?', 'zal( .+ | )fout gaan', 'iedereen zal( .+ | )denken', '(zal|gaat) niet stoppen', 'waardeloos in', 'een volslagen', 'zwart of wit', 'geef mij de schuld', 'z(ij|e) zullen( .+ | )niet geloven', 'z(ij|e) zal( .+ | )geloven', 'een toxische?', 'hij zal( .+ | )weten', '(zal|gaat) nooit stoppen', 'alleen het (slechte|verkeerde)', 'een ongewenste?', 'niemand zal( .+ | )denken', 'niemand zal( .+ | )weten', 'j(ij|e) (zal|zult)( .+ | )weten', 'een giftige?', 'z(ij|e) zullen( .+ | )geloven', 'iedereen gelooft', 'w(ij|e) geloven( .+ | )niet', 'een hulpelo(os|ze)', 'z(ij|e) zal( .+ | )weten', 'het enige (w|d)at ik zie', 'ik ben verantwoordelijk', 'niemand zal( .+ | )geloven', 'hij zal( .+ | )denken', 'een absolu(ut|te)', 'z(ij|e) zal( .+ | )denken', 'irrelevant', 'een afschuwelijke?', 'gebeurt (elke|iedere) keer', 'mij de schuld geven', 'w(ij|e) geloven', 'niet echt top', 'niet echt geweldig', 'z(ij|e) zullen( .+ | )weten', '(een )?ongeliefde?', 'een kapotte', '(he|al)lemaal door mij', 'top,? maar', 'voelt nog steeds', 'telt niet', 'niemand denkt', 'gaat wel,? maar', 'ik voel m(ij|e) verantwoordelijk', 'hij gelooft( .+ | )niet', 'z(ij|e) zullen( .+ | )denken', 'gebeurt altijd', 'geweldig,? maar', 'een lelijke?', 'z(e|ij) zijn een']
    cds_col_remained = []
    for col in cds_names:
        if col not in common_cds: 
            cds_col_remained.append(col)
            df_all[col] = 0

    all_common_indexes = []
    for y in years:
        print("year =",y)
        events =  os.listdir(image_folder_path+y)

        for e in events:
            
            df = pd.read_csv(image_folder_path+y+'/'+e+'/'+"CDS_results.tsv",sep= '\t')
            df.drop(common_cds,axis=1,inplace=True)
            
            df['id'] = df['event_id'].astype("string") + df['message_id'].astype("string")
            df.set_index("id",inplace=True)
            df.drop(['message_id'],axis=1,inplace=True)
            
            df_common_indexes = df.index.intersection(df_all.index)
            if len(df_common_indexes) == 0: continue
            all_common_indexes.extend(df_common_indexes)
            
            df_all.loc[df_common_indexes,cds_col_remained] = df.loc[df_common_indexes,cds_col_remained]
        
            
        print("-------------------------------------------------------")
        
    print("all_common_indexes =",len(all_common_indexes))
    print("all_indexes =",len(df_all.index))
    non_common_indexes = list(set(df_all.index) - set(all_common_indexes))
    df_all.drop(non_common_indexes,inplace=True)
    
    return df_all
df_all = combine_with_cds_data(df_all)
                

In [57]:
df_all.to_csv("df_all_prechat.tsv",sep='\t')

In [34]:
df_all = get_msg_info_data()
df_post = pd.read_csv(image_folder_path+"postchat_questions.tsv",sep= '\t').dropna()
df_all = combine_with_chatquestions_data(df_all,df_post)
df_all = combine_with_cds_data(df_all)


year = 2017
-------------------------------------------------------
year = 2018
-------------------------------------------------------
year = 2019
-------------------------------------------------------
year = 2020
-------------------------------------------------------
year = 2021
-------------------------------------------------------
all_common_indexes = 371753
all_indexes = 371753


In [35]:
df_all.to_csv("df_all_postchat.tsv",sep='\t')

In [36]:
df_all

,postive,negative,user_handle,sec_since_start,event_id,ik ben een,hij is een,z(e|ij) is een,(het|'t) is een,d(a|i)t is een,...,(door|vanwege) mijn,(door|vanwege) mij,ik heb( .+ | )veroorzaakt,allemaal mijn (schuld|fout),mijn fout,mijn verantwoordelijkheid,zou(den)? moeten,moet(en)?,had(den)? moeten,heb(ben)? moeten
id,,,,,,,,,,,,,,,,,,,,,
cl0000031,4.0,4.500,employee,8.0,cl000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl0000032,4.0,4.500,client,35.0,cl000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl0000033,4.0,4.500,client,200.0,cl000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl0000034,4.0,4.500,employee,380.0,cl000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl0000035,4.0,4.500,employee,408.0,cl000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cl024847169,3.0,5.625,client,5839.0,cl024847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl024847170,3.0,5.625,employee,5848.0,cl024847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cl024847171,3.0,5.625,client,5865.0,cl024847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
